In [1]:
%pylab inline

import os
from src.CAPTCHA_to_signs import DFS

Populating the interactive namespace from numpy and matplotlib


## Preprocess

W tym miejscu chcemy podzielić surowe CAPTCHA ze znanymi etykietami na poszczególne znaki z również znanymi etykietami. Powstały zbiór wykorzystamy do nauki modelu

In [2]:
root_path = os.getcwd()
img_dir_path = os.path.join(root_path, 'data', 'raw')
invalid_img_dir_path = os.path.join(root_path, 'data', 'invalid')
signs_output_dir_path = os.path.join(root_path, 'data', 'signs')

In [3]:
for sign in '23456789ABCDEFGHJKLMNPQRSTUVWXYZ':
    sign_dir_path = os.path.join(signs_output_dir_path, sign)
    
    if not os.path.exists(sign_dir_path):
        os.makedirs(sign_dir_path)

In [4]:
from collections import Counter

counter = Counter()

for img_filename in os.listdir(img_dir_path):
    img_path = os.path.join(img_dir_path, img_filename)
    
    label = img_filename.split('.')[0].upper()
    signs = DFS(img_path).dfs_all()
    
    # Przenieś niepoprawne obrazki do folderu invalid
    if len(label) != len(signs):
        invalid_img_path = os.path.join(invalid_img_dir_path, img_filename)
        os.rename(img_path, invalid_img_path)
        continue
        
    for sign, target in zip(signs, label):
        counter[target] += 1
        
        sign_path = os.path.join(signs_output_dir_path, target, str(counter[target]))
        sign.save(sign_path)

## Model

In [32]:
import keras
from keras.datasets import mnist
from keras.preprocessing import image
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

In [8]:
signs_dir_path = os.path.join(root_path, 'data', 'signs')

In [18]:
X, y = [], []

for sign in '23456789ABCDEFGHJKLMNPQRSTUVWXYZ':
    sign_dir_path = os.path.join(signs_dir_path, sign)
    
    for img_filename in os.listdir(sign_dir_path):
        img_path = os.path.join(sign_dir_path, img_filename)
        
        img = image.load_img(img_path, grayscale=True)
        img = image.img_to_array(img)
        
        X.append(img)
        y.append(sign)
        
X, y = np.array(X), np.array(y)

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [27]:
batch_size = 128
num_classes = len('23456789ABCDEFGHJKLMNPQRSTUVWXYZ')
epochs = 15

In [36]:
img_rows, img_cols = 16, 16
input_shape = (img_rows, img_cols, 1)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

In [30]:
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

('X_train shape:', (26729, 16, 16, 1))
(26729, 'train samples')
(6683, 'test samples')


In [33]:
encoder = LabelBinarizer()
y_train = encoder.fit_transform(y_train)
y_test = encoder.fit_transform(y_test)

In [37]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

In [38]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [39]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, y_test))

Train on 26729 samples, validate on 6683 samples
Epoch 1/15
26729/26729 [==============================] - 6s 233us/step - loss: 14.8476 - acc: 0.0699 - val_loss: 12.2347 - val_acc: 0.2256
Epoch 2/15
26729/26729 [==============================] - 6s 222us/step - loss: 4.1426 - acc: 0.4319 - val_loss: 0.1386 - val_acc: 0.9740
Epoch 3/15
26729/26729 [==============================] - 6s 222us/step - loss: 0.4845 - acc: 0.8498 - val_loss: 0.0060 - val_acc: 0.9991
Epoch 4/15
26729/26729 [==============================] - 6s 226us/step - loss: 0.2030 - acc: 0.9348 - val_loss: 7.9704e-04 - val_acc: 0.9997
Epoch 5/15
26729/26729 [==============================] - 6s 223us/step - loss: 0.1177 - acc: 0.9608 - val_loss: 9.5842e-05 - val_acc: 1.0000
Epoch 6/15
26729/26729 [==============================] - 6s 222us/step - loss: 0.0834 - acc: 0.9729 - val_loss: 3.2829e-05 - val_acc: 1.0000
Epoch 7/15
26729/26729 [==============================] - 6s 227us/step - loss: 0.0661 - acc: 0.9779 - val_lo

In [41]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

('Test loss:', 1.6634578938223013e-07)
('Test accuracy:', 1.0)


## Predict

In [69]:
correct = 0

for img_filename in os.listdir(img_dir_path):
    img_path = os.path.join(img_dir_path, img_filename)
    
    label = img_filename.split('.')[0].upper()
    signs = DFS(img_path).dfs_all()
    
    signs = np.array([s.bitmap * 255 for s in signs]).reshape(-1, 16, 16, 1)
    predict = model.predict_classes(signs)
    
    if label == ''.join(encoder.classes_[predict]):
        correct += 1

In [72]:
float(correct) / len(os.listdir(img_dir_path))

1.0